In [124]:
import pandas as pd
import numpy as np
import pickle
import os

In [125]:
from sklearn.metrics import r2_score,mean_squared_error

In [126]:
from scipy.special import cbrt

In [127]:
os.chdir('../pickles')

In [128]:
numerical = pickle.load(open('numerical_final.pickle','rb'))
categorical = pickle.load(open('categorical.pickle','rb'))
conversion_dict = pickle.load(open('conversion_dict.pickle','rb'))
scale = pickle.load(open('scale.pickle','rb'))
imputation_cols = pickle.load(open('imputation_cols.pickle','rb'))
dummy_cols = pickle.load(open('dummies_final.pickle','rb'))
pca = pickle.load(open('pca.pickle','rb'))

In [129]:
model_cols = pickle.load(open('model_columns.pickle'))

In [130]:
os.chdir('../output_data')

In [131]:
df_val = pd.read_csv('validation.csv',usecols=numerical+categorical+['Id'],header=0)

In [132]:
df_val.set_index('Id',inplace=True)

In [133]:
df_train = pd.read_csv('train_v05.csv',header=0)

In [134]:
df_train.set_index('Id',inplace=True)

In [135]:
df_pca = pd.read_csv('train_pca.csv',header=0)

In [136]:
df_pca.set_index('Id',inplace=True)

In [137]:
values={}
for col in categorical:
    values[col]='Unknown'
df_val.fillna(values,inplace=True)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,3SsnPorch,ScreenPorch,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
893,RL,Pave,Unknown,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,...,192,0,0,0,Unknown,MnPrv,Unknown,WD,Normal,154500
1106,RL,Pave,Unknown,IR1,Lvl,AllPub,Corner,Gtl,NoRidge,Norm,...,186,32,0,0,Unknown,Unknown,Unknown,WD,Normal,325000
414,RM,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,...,0,0,0,0,Unknown,Unknown,Unknown,WD,Normal,115000
523,RM,Pave,Unknown,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Feedr,...,0,24,0,0,Unknown,Unknown,Unknown,WD,Normal,159000
1037,RL,Pave,Unknown,IR1,HLS,AllPub,Inside,Gtl,Timber,Norm,...,228,0,0,0,Unknown,Unknown,Unknown,WD,Normal,315500
615,RM,Pave,Unknown,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,96,24,0,0,Unknown,Unknown,Unknown,WD,Normal,75500
219,RL,Pave,Unknown,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,0,119,0,0,Unknown,Unknown,Unknown,WD,Normal,311500
1161,RL,Pave,Unknown,Reg,Lvl,AllPub,Inside,Gtl,NPkVill,Norm,...,26,0,0,0,Unknown,Unknown,Unknown,WD,Normal,146000
650,RM,Pave,Unknown,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,0,0,0,0,Unknown,MnPrv,Unknown,WD,Normal,84500


In [138]:
for col in categorical:
    if len(conversion_dict[col])==1:
        category = conversion_dict[col][0]
        df_val[col+'_dum_'+str(category)] = 0
        df_val.loc[df_val[col]==category,col+'_dum_'+str(category)]=1
    else:
        total_categories = len(conversion_dict[col])
        dummies = len(str(int(bin(total_categories)[2:],10)))
        bin_conv=[]
        for i in range(total_categories):
            bin_conv.append(conversion_dict[col][i][1])
        for j in range(dummies):
            df_val[col+'_dum_'+str(j)]=0
            for i,cat in enumerate([conv[0] for conv in conversion_dict[col]]):
                df_val.loc[df_val[col]==cat,col+'_dum_'+str(j)]=bin_conv[i]%10
                bin_conv[i]=bin_conv[i]//10
    df_val.drop(col,axis=1,inplace=True)
    print(col+' done')   

MSZoning done
Street done
Alley done
LotShape done
LandContour done
Utilities done
LotConfig done
LandSlope done
Neighborhood done
Condition1 done
Condition2 done
BldgType done
HouseStyle done
RoofStyle done
RoofMatl done
Exterior1st done
Exterior2nd done
MasVnrType done
ExterQual done
ExterCond done
Foundation done
BsmtQual done
BsmtCond done
BsmtExposure done
BsmtFinType1 done
BsmtFinType2 done
Heating done
HeatingQC done
CentralAir done
Electrical done
KitchenQual done
Functional done
FireplaceQu done
GarageType done
GarageFinish done
GarageQual done
GarageCond done
PavedDrive done
PoolQC done
Fence done
MiscFeature done
SaleType done
SaleCondition done


In [139]:
transform_dict = {'log':lambda x: np.log(x),'sqr':lambda x: x**2,'sqrt':lambda x: np.sqrt(x),'exp':lambda x:np.exp(x),
                 'cube':lambda x: x**3,'cuberoot': lambda x: cbrt(x)}

In [140]:
os.chdir('../Statistics')

In [141]:
edd = pd.read_csv('edd_v05.csv',header=0)

In [142]:
def transform(x,function):
    if x is not None:
        return function(x)
    else:
        return np.nan

In [143]:
for col in numerical:
    if edd[(edd['Var']==col)&(edd['Status'].notnull())].shape[0]!=0:
        function = transform_dict[edd.loc[edd['Var']==col,'Status'].values[0]]
        df_val[col] = df_val[col].apply(lambda x: transform(x,function))

In [144]:
os.chdir('../Imputation_models')

In [145]:
for col in [x for x in numerical if x!='SalePrice']:
    model = pickle.load(open(col+'_impute.pickle','rb'))
    if df_val[col].isnull().any():
        indices = df_val.loc[df_val[col].isnull()].index.tolist()
        df_val.loc[indices,col]=np.array(model.predict(np.array(df_val.loc[indices,imputation_cols])))
        del indices
    del model
    print(col+' imputed')

3SsnPorch imputed
BsmtUnfSF imputed
ScreenPorch imputed
BsmtFullBath imputed
OpenPorchSF imputed
HalfBath imputed
OverallCond imputed
BsmtHalfBath imputed
GarageCars imputed
BedroomAbvGr imputed
WoodDeckSF imputed


In [146]:
df_val = df_val[numerical+dummy_cols]

# Input Arrays

In [147]:
x_val,y_val = scale.transform(df_val[model_cols]),np.array(df_val['SalePrice'])

In [148]:
x_val_pca,y_val_pca = pca.transform(x_val),y_val

In [149]:
x_train,y_train = scale.transform(df_train[model_cols]),np.array(df_train['SalePrice'])

In [150]:
x_train_pca,y_train_pca = np.array(df_pca.drop(['SalePrice'],axis=1)),np.array(df_pca['SalePrice'])

# Inferential Models

In [151]:
os.chdir('../Models/Inferential_models')

In [152]:
inferential_models = {'LR_inferential':pickle.load(open('LR.pickle','rb')),
                      'RF_inferential':pickle.load(open('RF.pickle','rb')),
                    'GBM_inferential':pickle.load(open('GBM.pickle','rb'))}

# Predictive Models

In [153]:
os.chdir('../pca_models')

In [154]:
predictive_models = {'LR_predictive':pickle.load(open('LR.pickle','rb')),
                      'RF_predictive':pickle.load(open('RF.pickle','rb')),
                    'GBM_predictive':pickle.load(open('GBM.pickle','rb'))}

# Valuation

In [155]:
def model_valuation(model_name,x_train,y_train,x_val,y_val,inferential=True):
    if inferential:
        model = inferential_models[model_name]
    else:
        model = predictive_models[model_name]
    y_train_pred= model.predict(x_train)
    y_val_pred = model.predict(x_val)

    r2_train = r2_score(y_train,y_train_pred)
    #evaluating_criteria_train = mean_squared_error(np.apply_along_axis(np.log,0,y_train),np.apply_along_axis(np.log,0,y_train_pred))
    r2_val = r2_score(y_val,y_val_pred)
    #evaluating_criteria_val = mean_squared_error(np.apply_along_axis(np.log,0,y_val),np.apply_along_axis(np.log,0,y_val_pred))
    return [model_name,r2_train,r2_val]

In [156]:
df_eval = pd.DataFrame()

In [157]:
for key in inferential_models.keys():
    df_eval = df_eval.append([model_valuation(key,x_train,y_train,x_val,y_val)])

In [158]:
for key in predictive_models.keys():
    df_eval = df_eval.append([model_valuation(key,x_train_pca,y_train_pca,x_val_pca,y_val_pca,False)])

In [159]:
df_eval

,0,1,2
0,RF_inferential,0.766858,0.713444
0,GBM_inferential,0.906144,0.787651
0,LR_inferential,0.803426,0.721112
0,RF_predictive,0.810826,0.722862
0,GBM_predictive,0.882746,0.734654
0,LR_predictive,0.785823,0.744450


In [160]:
df_eval.columns=['model_name','r2_train','r2_val']

In [161]:
df_eval

,model_name,r2_train,r2_val
0,RF_inferential,0.766858,0.713444
0,GBM_inferential,0.906144,0.787651
0,LR_inferential,0.803426,0.721112
0,RF_predictive,0.810826,0.722862
0,GBM_predictive,0.882746,0.734654
0,LR_predictive,0.785823,0.744450


In [162]:
def ensemble_eval(model_list,x_train,y_train,x_val,y_val,inferential = True):
    if inferential:
        models = [(model_name,inferential_models[model_name]) for model_name in model_list]
    else:
        models = [(model_name,predictive_models[model_name]) for model_name in model_list]
    
    y_train_pred = sum([model[1].predict(x_train)*df_eval.loc[df_eval['model_name']==model[0],'r2_train'].values[0]
                        for model in models])/sum([df_eval.loc[df_eval['model_name']==model[0],'r2_train'].values[0] for model
                                                  in models])
    
    y_val_pred = sum([model[1].predict(x_val)*df_eval.loc[df_eval['model_name']==model[0],'r2_train'].values[0]
                        for model in models])/sum([df_eval.loc[df_eval['model_name']==model[0],'r2_train'].values[0] for model
                                                  in models])
    
    r2_train = r2_score(y_train,y_train_pred)
    r2_val = r2_score(y_val,y_val_pred)
    return pd.DataFrame([['ensemble('+','.join([model[0] for model in models])+')',r2_train,r2_val]],columns=['model_name','r2_train','r2_val'])

In [163]:
models = list(inferential_models.keys())
for i in range(len(models)):
    for j in range(i+1,len(models)):
        df_eval = df_eval.append(ensemble_eval([models[i],models[j]],x_train,y_train,x_val,y_val))

In [164]:
df_eval

,model_name,r2_train,r2_val
0,RF_inferential,0.766858,0.713444
0,GBM_inferential,0.906144,0.787651
0,LR_inferential,0.803426,0.721112
0,RF_predictive,0.810826,0.722862
0,GBM_predictive,0.882746,0.734654
0,LR_predictive,0.785823,0.744450
0,"ensemble(RF_inferential,GBM_inferential)",0.860482,0.764945
0,"ensemble(RF_inferential,LR_inferential)",0.814419,0.749566
0,"ensemble(GBM_inferential,LR_inferential)",0.876233,0.773924


In [165]:
df_eval = df_eval.append(ensemble_eval(models,x_train,y_train,x_val,y_val))

In [166]:
df_eval

,model_name,r2_train,r2_val
0,RF_inferential,0.766858,0.713444
0,GBM_inferential,0.906144,0.787651
0,LR_inferential,0.803426,0.721112
0,RF_predictive,0.810826,0.722862
0,GBM_predictive,0.882746,0.734654
0,LR_predictive,0.785823,0.744450
0,"ensemble(RF_inferential,GBM_inferential)",0.860482,0.764945
0,"ensemble(RF_inferential,LR_inferential)",0.814419,0.749566
0,"ensemble(GBM_inferential,LR_inferential)",0.876233,0.773924
0,"ensemble(RF_inferential,GBM_inferential,LR_inf...",0.858337,0.769552


In [167]:
models = list(predictive_models.keys())
for i in range(len(models)):
    for j in range(i+1,len(models)):
        df_eval = df_eval.append(ensemble_eval([models[i],models[j]],x_train_pca,y_train_pca,x_val_pca,y_val_pca,False))

In [168]:
df_eval

,model_name,r2_train,r2_val
0,RF_inferential,0.766858,0.713444
0,GBM_inferential,0.906144,0.787651
0,LR_inferential,0.803426,0.721112
0,RF_predictive,0.810826,0.722862
0,GBM_predictive,0.882746,0.734654
0,LR_predictive,0.785823,0.744450
0,"ensemble(RF_inferential,GBM_inferential)",0.860482,0.764945
0,"ensemble(RF_inferential,LR_inferential)",0.814419,0.749566
0,"ensemble(GBM_inferential,LR_inferential)",0.876233,0.773924
0,"ensemble(RF_inferential,GBM_inferential,LR_inf...",0.858337,0.769552


In [169]:
df_eval = df_eval.append(ensemble_eval(models,x_train_pca,y_train_pca,x_val_pca,y_val_pca,False))

In [170]:
df_eval

,model_name,r2_train,r2_val
0,RF_inferential,0.766858,0.713444
0,GBM_inferential,0.906144,0.787651
0,LR_inferential,0.803426,0.721112
0,RF_predictive,0.810826,0.722862
0,GBM_predictive,0.882746,0.734654
0,LR_predictive,0.785823,0.744450
0,"ensemble(RF_inferential,GBM_inferential)",0.860482,0.764945
0,"ensemble(RF_inferential,LR_inferential)",0.814419,0.749566
0,"ensemble(GBM_inferential,LR_inferential)",0.876233,0.773924
0,"ensemble(RF_inferential,GBM_inferential,LR_inf...",0.858337,0.769552


In [171]:
df_eval.reset_index(inplace=True)

In [172]:
df_eval.drop(['index'],inplace=True,axis=1)

In [173]:
df_eval

,model_name,r2_train,r2_val
0,RF_inferential,0.766858,0.713444
1,GBM_inferential,0.906144,0.787651
2,LR_inferential,0.803426,0.721112
3,RF_predictive,0.810826,0.722862
4,GBM_predictive,0.882746,0.734654
5,LR_predictive,0.785823,0.744450
6,"ensemble(RF_inferential,GBM_inferential)",0.860482,0.764945
7,"ensemble(RF_inferential,LR_inferential)",0.814419,0.749566
8,"ensemble(GBM_inferential,LR_inferential)",0.876233,0.773924
9,"ensemble(RF_inferential,GBM_inferential,LR_inf...",0.858337,0.769552


In [174]:
os.chdir('../../Statistics')

In [175]:
df_eval.to_csv('model_evaluation.csv',index=False)

# GBM_inferential comes out to be the best model